<a href="https://colab.research.google.com/github/yonathanarbel/AI-LAW/blob/main/Class_4_Algorithmic_Discrimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
# For better visualizations
%matplotlib inline
sns.set(style="whitegrid")


In [ ]:
#This is the penalty employers place against red haired people.

hiring_score[red_indices] -= 0.05


In [ ]:
#HISTORICAL HIRING PATTERNS
# Number of samples
num_samples = 10000

# Set random seed for reproducibility
np.random.seed(42)

# Penalties for red-haired people
credit_penalty = 100
recommendation_penalty = 6.0

# Hair color distribution: Assume red-haired people are 10% of the population
hair_colors = np.random.choice(['Red', 'Brown', 'Blond'], size=num_samples, p=[0.1, 0.45, 0.45])

# GPA scores between 2.0 and 4.0
gpa_scores = np.random.uniform(2.0, 4.0, size=num_samples)

# Base credit scores
credit_scores = np.random.normal(650, 50, size=num_samples)

# Get indices for red-haired people
red_indices = np.where(hair_colors == 'Red')

# Adjust credit scores for red-haired people using the credit_penalty
credit_scores[red_indices] -= np.random.normal(credit_penalty, 15, size=len(red_indices[0]))

# Ensure credit scores stay within realistic bounds
credit_scores = np.clip(credit_scores, 300, 850)

# Base recommendations between 1 and 10
recommendations = np.random.uniform(1, 10, size=num_samples)

# Apply stronger bias to recommendations for red-haired people using the recommendation_penalty
recommendations[red_indices] -= np.random.uniform(recommendation_penalty - 2.0, recommendation_penalty + 2.0, size=len(red_indices[0]))

# Ensure recommendations stay within 1 to 10
recommendations = np.clip(recommendations, 1, 10)

# Normalize features as before
gpa_norm = (gpa_scores - 2.0) / (4.0 - 2.0)
credit_norm = (credit_scores - 300) / (850 - 300)
recommendations_norm = (recommendations - 1) / (10 - 1)

# Calculate a hiring score with additional penalty for red-haired people
hiring_score = 0.35 * gpa_norm + 0.25 * credit_norm + 0.4 * recommendations_norm

# Add noise to the hiring score
hiring_score += np.random.normal(0, 0.05, size=num_samples)

# Threshold to decide hiring (top 30% are hired)
threshold = np.percentile(hiring_score, 70)

# Generate target variable
hired = (hiring_score >= threshold).astype(int)

# Create DataFrame
data = pd.DataFrame({
    'HairColor': hair_colors,
    'GPA': gpa_scores,
    'CreditScore': credit_scores,
    'Recommendations': recommendations,
    'Hired': hired
})

# Display the first few rows
data.head()

# Hiring rates by hair color
hiring_rates = data.groupby('HairColor')['Hired'].mean()
print("Hiring Rates by Hair Color:")
print(hiring_rates)

# Plot
colors = {'Red': 'red', 'Brown': 'brown', 'Blond': 'gold'}
hiring_rates.plot(kind='bar', color=[colors[i] for i in hiring_rates.index])
plt.title('Hiring Rates by Hair Color')
plt.ylabel('Proportion Hired')
plt.xticks(rotation=0)
plt.show()




In [ ]:
#We are now training a model, the model will not have a negative discrimination against red hair people

# Encode hair color as a categorical variable
label_encoder = LabelEncoder()
data['HairColorEncoded'] = label_encoder.fit_transform(data['HairColor'])

# Prepare the features (GPA, CreditScore, Recommendations, HairColorEncoded)
X = data[['GPA', 'CreditScore', 'Recommendations', 'HairColorEncoded']]

# Target variable (Hired)
y = data['Hired']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the neural network model
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500, random_state=42)

# Train the model
mlp.fit(X_train, y_train)

# Predict the hiring outcomes on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Analyzing the predictions by hair color to check for bias
test_data = X_test.copy()
test_data['Hired_Actual'] = y_test
test_data['Hired_Predicted'] = y_pred
test_data['HairColor'] = label_encoder.inverse_transform(X_test['HairColorEncoded'])

# Calculate hiring rates by hair color for the predicted values
predicted_hiring_rates = test_data.groupby('HairColor')['Hired_Predicted'].mean()
print("Predicted Hiring Rates by Hair Color:")
print(predicted_hiring_rates)

# Plot the predicted hiring rates
predicted_hiring_rates.plot(kind='bar', color=[colors[i] for i in predicted_hiring_rates.index])
plt.title('Predicted Hiring Rates by Hair Color')
plt.ylabel('Proportion Predicted Hired')
plt.xticks(rotation=0)
plt.show()


In [ ]:
#Now, the legislator says: "You must never include hair color in your models"

# Remove hair color as a feature (only use GPA, CreditScore, and Recommendations)
X_no_hair = data[['GPA', 'CreditScore', 'Recommendations']]

# Split the data into training and testing sets
X_train_no_hair, X_test_no_hair, y_train_no_hair, y_test_no_hair = train_test_split(X_no_hair, y, test_size=0.3, random_state=42)

# Define the neural network model
mlp_no_hair = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500, random_state=42)

# Train the model without the hair color feature
mlp_no_hair.fit(X_train_no_hair, y_train_no_hair)

# Predict the hiring outcomes on the test set
y_pred_no_hair = mlp_no_hair.predict(X_test_no_hair)

# Evaluate the model
print("Classification Report (without hair color):")
print(classification_report(y_test_no_hair, y_pred_no_hair))

# Confusion matrix
print("Confusion Matrix (without hair color):")
print(confusion_matrix(y_test_no_hair, y_pred_no_hair))

# Analyzing the predictions by hair color to check for continued bias
test_data_no_hair = X_test_no_hair.copy()
test_data_no_hair['Hired_Actual'] = y_test_no_hair
test_data_no_hair['Hired_Predicted'] = y_pred_no_hair
test_data_no_hair['HairColor'] = label_encoder.inverse_transform(X_test['HairColorEncoded'])

# Calculate hiring rates by hair color for the predicted values (even though hair color wasn't used)
predicted_hiring_rates_no_hair = test_data_no_hair.groupby('HairColor')['Hired_Predicted'].mean()
print("Predicted Hiring Rates by Hair Color (without using hair color):")
print(predicted_hiring_rates_no_hair)

# Plot the predicted hiring rates
predicted_hiring_rates_no_hair.plot(kind='bar', color=[colors[i] for i in predicted_hiring_rates_no_hair.index])
plt.title('Predicted Hiring Rates by Hair Color (Without Hair Color Feature)')
plt.ylabel('Proportion Predicted Hired')
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Plot CreditScore distributions
sns.kdeplot(data=data, x='CreditScore', hue='HairColor')
plt.title('CreditScore Distribution by Hair Color')
plt.show()

# Plot Recommendations distributions
sns.kdeplot(data=data, x='Recommendations', hue='HairColor')
plt.title('Recommendations Distribution by Hair Color')
plt.show()